
Importing all the Libraries that are needed

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series

from sklearn.utils import class_weight
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif, chi2
import numpy as np



In [4]:
#Reading all datasets available

train_set = pd.read_csv('/content/drive/MyDrive/ML/Data Sets/speech-based-classification-layer-7/train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/ML/Data Sets/speech-based-classification-layer-7/test.csv')
valid_set = pd.read_csv('/content/drive/MyDrive/ML/Data Sets/speech-based-classification-layer-7/valid.csv')


In [5]:
#Checking whether there are coloumns with missing values

null_val_cols = train_set.columns[train_set.isnull().any()]

for col in null_val_cols:
    print(col)

label_2


In [67]:
#Removing rows with unavailable values in label_2 in both train and valid data sets

train_set = train_set.dropna(subset=['label_2'])
valid_set = valid_set.dropna(subset=['label_2'])
#train_set.to_csv('/content/drive/MyDrive/ML/Data Sets/speech-based-classification-layer-12/cleaned_train_set.csv', index=False)
train_set.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
480,0.083870,0.166150,-0.034322,-0.075919,-0.171775,0.012949,0.165340,-0.106969,0.027650,-0.024831,...,-0.052727,0.033423,-0.054580,-0.052529,-0.117777,-0.033383,5,25.0,1,6
481,0.071744,-0.016584,-0.039873,-0.081580,-0.159366,-0.066516,0.061793,-0.116348,0.097517,-0.085288,...,-0.123597,-0.076431,0.040986,0.015559,-0.036971,0.165624,5,25.0,1,6
482,0.210556,-0.000793,0.081505,-0.071365,-0.167036,0.036712,0.143173,-0.074924,-0.007206,-0.143535,...,-0.067351,-0.037610,0.007519,0.046775,0.068004,0.017755,5,25.0,1,6
483,0.098933,-0.022292,0.032271,-0.012868,-0.164382,-0.095997,0.195600,0.010783,-0.017376,-0.034254,...,-0.043167,0.005910,-0.081875,0.082903,0.081041,0.022018,5,25.0,1,6
484,0.004612,0.075626,-0.010036,-0.029415,-0.054140,-0.000785,0.085094,-0.050050,0.075547,-0.055219,...,-0.004819,-0.041974,0.022569,0.063767,-0.044779,0.062611,5,25.0,1,6


In [68]:
#Formating the datasets in the way we want. Features and labelts are being seperated and done seperately for both Train and valid sets
classifier = SVC()

X = pd.DataFrame(train_set.iloc[:, :-4].values)
label_1 = train_set['label_1']
label_2 = train_set['label_2']
label_3 = train_set['label_3']
label_4 = train_set['label_4']

X_valid = pd.DataFrame(valid_set.iloc[:, :-4].values)
label_1_valid = valid_set['label_1']
label_2_valid = valid_set['label_2']
label_3_valid = valid_set['label_3']
label_4_valid = valid_set['label_4']

#Initial accuracy is being checked after training through a svc model.

classifier.fit(X,label_1)
predicted_label1 = classifier.predict(X_valid)

print("accuracy_score: ",metrics.accuracy_score(label_1_valid, predicted_label1))
print("f1_score: ",f1_score(label_1_valid, predicted_label1, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(label_1_valid, predicted_label1, average='weighted' ))
print("recall_score: ",metrics.recall_score(label_1_valid, predicted_label1, average='weighted'))


accuracy_score:  0.9578804347826086
f1_score:  0.958863763969804
precision_score:  0.9659790263182586
recall_score:  0.9578804347826086


###Label 1

In [8]:
# @title Grid Search

#Trying out Grid search for parameter tuning.Takes a long time to porvide a good solution.

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma='auto'),{
    'C': [1,10,20],
    'kernel':['rbf','linear']
}, cv=5, return_train_score=False)

clf.fit(X,label_1)
clf.cv_results_

{'mean_fit_time': array([276.7382236 ,  36.07352719, 121.2159224 ,  31.06083694,
         89.258354  ,  30.99198766]),
 'std_fit_time': array([1.73471667, 0.30717987, 0.55391181, 0.27634528, 0.43896617,
        0.31075905]),
 'mean_score_time': array([149.4195734 ,  66.37185683, 142.87124181,  61.50486188,
        132.00097065,  60.70594659]),
 'std_score_time': array([1.14305674, 3.41359039, 2.5736771 , 1.11341683, 0.76011462,
        1.25514653]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [13]:
# @title Grid Search results
df = pd.DataFrame(clf.cv_results_)
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.402104
1,1,linear,0.966476
2,10,rbf,0.904672
3,10,linear,0.960378
4,20,rbf,0.930385
5,20,linear,0.959843


In [16]:
# @title Grid Search Best Score
clf.best_score_


0.9664764621968616

In [17]:
# @title Grid Search Best Params
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [51]:
# @title SVC classifier with the best Parameters
classifier = SVC(C=1, kernel='linear')
classifier.fit(X,label_1)

predicted_label1 = classifier.predict(X_valid)

print("accuracy_score: ",metrics.accuracy_score(label_1_valid, predicted_label1))
print("f1_score: ",f1_score(label_1_valid, predicted_label1, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(label_1_valid, predicted_label1, average='weighted' ))
print("recall_score: ",metrics.recall_score(label_1_valid, predicted_label1, average='weighted'))





accuracy_score:  0.9573333333333334
f1_score:  0.9493557742842689
precision_score:  0.9447065871635532
recall_score:  0.9573333333333334


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
print(X.shape)  # Check the shape of X_train
print(test_set.shape)   # Check the shape of X_test

(28040, 768)
(744, 769)


In [55]:
# @title Making a csv with the test results
#Since svm gave the best accuracy

X_test = pd.DataFrame(test_set.iloc[:, 1:].values)
predicted_label_test = classifier.predict(X_test)

predicted_df = pd.DataFrame({'label_1': predicted_label_test})

# Save the DataFrame to a CSV file
predicted_df.to_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label1.csv', index=False)


In [23]:
selector = SelectKBest(f_classif, k=200)
x_new = selector.fit_transform(X, label_1)
x_valid_df_after_f_classif = selector.transform(X_valid)

classifier = SVC(C=1, kernel='linear')
classifier.fit(x_new,label_1)
predicted_label1 = classifier.predict(x_valid_df_after_f_classif)

print("accuracy_score: ",metrics.accuracy_score(label_1_valid, predicted_label1))
print("f1_score: ",f1_score(label_1_valid, predicted_label1, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(label_1_valid, predicted_label1, average='weighted' ))
print("recall_score: ",metrics.recall_score(label_1_valid, predicted_label1, average='weighted'))

accuracy_score:  0.9453333333333334
f1_score:  0.9370716435107725
precision_score:  0.9329798558739735
recall_score:  0.9453333333333334


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
# @title PCA on the datasets
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95, svd_solver='full')
pca.fit(X)
x_train_trf = pd.DataFrame(pca.transform(X))
x_valid_trf_pca = pd.DataFrame(pca.transform(X_valid))
print('Shape after PCA: ',x_train_trf.shape)


Shape after PCA:  (28040, 238)


In [25]:
# @title using the classifier to check accuracy

classifier = svm.SVC(C=1, kernel='linear')
classifier.fit(x_train_trf,label_1)
predicted_label1 = classifier.predict(x_valid_trf_pca)

print("accuracy_score: ",metrics.accuracy_score(label_1_valid, predicted_label1))
print("f1_score: ",f1_score(label_1_valid, predicted_label1, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(label_1_valid, predicted_label1, average='weighted' ))
print("recall_score: ",metrics.recall_score(label_1_valid, predicted_label1, average='weighted'))

accuracy_score:  0.9466666666666667
f1_score:  0.9387835708594621
precision_score:  0.9350008292475784
recall_score:  0.9466666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# @title svm classifier with parameters
classifier = SVC(C=1, kernel='linear')
classifier.fit(X,label_1)

predicted_label1 = classifier.predict(X_valid)

print("accuracy_score: ",metrics.accuracy_score(label_1_valid, predicted_label1))
print("f1_score: ",f1_score(label_1_valid, predicted_label1, average='weighted')) #f-1 score
print("precision_score: ",metrics.precision_score(label_1_valid, predicted_label1, average='weighted' ))
print("recall_score: ",metrics.recall_score(label_1_valid, predicted_label1, average='weighted'))

#Label 3

In [70]:
# @title PCA for Label 3
from sklearn.decomposition import PCA

#Validation
pca = PCA(n_components=0.95, svd_solver='full')
pca.fit(X)
x_train_pca = pd.DataFrame(pca.transform(X)) #train
x_valid_pca = pd.DataFrame(pca.transform(X_valid)) #valid
x_test_pca = pd.DataFrame(pca.transform(X_test))
print('Shape after PCA: ',x_train_pca.shape)


Shape after PCA:  (28040, 238)


In [57]:
# @title svm classifier without pca
from sklearn import svm

# For classification with a linear kernel
classifier = svm.SVC(kernel='linear', C=1)

# For regression with a linear kernel
# classifier = svm.SVR(kernel='linear', C=1.0)

# Train the model
# classifier.fit(x_train_df, y_train_df)
classifier.fit(X, label_3)

#predict for validation
y_valid_pred = classifier.predict(X_valid)

#accuracy
print("accuracy_score: ",metrics.accuracy_score(label_3_valid, y_valid_pred))

accuracy_score:  0.9986666666666667


In [44]:
# @title svm classifier with pca
from sklearn import svm

# For classification with a linear kernel
classifier = svm.SVC(kernel='linear', C=1)

# For regression with a linear kernel
# classifier = svm.SVR(kernel='linear', C=1.0)

# Train the model
# classifier.fit(x_train_df, y_train_df)
classifier.fit(x_train_pca, label_3)

#predict for validation
y_valid_pred = classifier.predict(x_valid_pca)

#accuracy
print("accuracy_score: ",metrics.accuracy_score(label_3_valid, y_valid_pred))

accuracy_score:  0.9973333333333333


In [45]:
# @title Label 3 random search
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint  # For defining parameter distributions
import numpy as np

param_dist = {
    'C': [100,10,1,0,0.1,0.01],
    'kernel': ['rbf','linear','poly','sigmoid'],
    'gamma': ['scale','auto'],
    'degree': [1,2,3,4],  # For the polynomial kernel
    'class_weight' : ['none','balanced']
}

svm = SVC()

random_search = RandomizedSearchCV(
    svm, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42, scoring='accuracy'
)

random_search.fit(x_train_pca, label_3)

best_params = random_search.best_params_
best_model = random_search.best_estimator_

print("best parameters:", best_params)

KeyboardInterrupt: ignored

In [ ]:
print("Shape of X:", x_train_pca.shape)
print("Shape of y:", label_3.shape)

In [ ]:
# @title Hyper parameter tuning
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold
from scipy.stats import uniform
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,HalvingGridSearchCV

# Define the hyperparameter grid
param_grid = {
    'C': [0.1,1,10,20,100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Experiment with different kernels
    'gamma':['scale', 'auto'],
    'degree': [1,2,3,4]

}

# Create an SVM classifier
svm_classifier = SVC(class_weight='balanced')

# Perform grid search with cross-validation
grid_search = HalvingGridSearchCV(svm_classifier, param_grid, scoring='accuracy', cv=5,n_jobs=-1, factor=2, verbose=1)
grid_search.fit(x_train_pca, label_3)

# Get the best model and hyperparameters
best_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the validation set
y_pred = best_svm_classifier.predict(x_valid_pca)

# Evaluate the model
print("Best Hyperparameters:", best_params)

In [60]:
#@title svm classifier with pca
from sklearn import svm
classifier_label3 = svm.SVC(kernel='rbf', C=10, gamma='auto', degree=1)

classifier_label3.fit(x_train_pca, label_3)

#predict for validation
y_valid_pred = classifier_label3.predict(x_valid_pca)

#accuracy
print("accuracy_score: ",metrics.accuracy_score(label_3_valid, y_valid_pred))

#predict
# y_test_pred = classifier.predict(x_test_df)
#y_test_predict_after_pca = classifier.predict(x_test_df_pca)


accuracy_score:  0.9973333333333333


In [63]:
#@title Making csv for label 3
predicted_label_test = classifier_label3.predict(x_test_pca)

predicted_df = pd.DataFrame({'label_3': predicted_label_test})

# Save the DataFrame to a CSV file
predicted_df.to_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label3.csv', index=False)

# Label 4


In [47]:
#@title svm Classifier without pca
from sklearn import svm

# For classification with a linear kernel
classifier = svm.SVC(kernel='linear', C=1)

# For regression with a linear kernel
# classifier = svm.SVR(kernel='linear', C=1.0)

# Train the model
# classifier.fit(x_train_df, y_train_df)
classifier.fit(X, label_4)

#predict for validation
y_valid_pred = classifier.predict(X_valid)

#accuracy
print("accuracy_score: ",metrics.accuracy_score(label_4_valid, y_valid_pred))

accuracy_score:  0.956


In [48]:
#@title svm classifier with cpa
from sklearn import svm

# For classification with a linear kernel
classifier = svm.SVC(kernel='linear', C=1)

# For regression with a linear kernel
# classifier = svm.SVR(kernel='linear', C=1.0)

# Train the model
# classifier.fit(x_train_df, y_train_df)
classifier.fit(x_train_pca, label_4)

#predict for validation
y_valid_pred = classifier.predict(x_valid_pca)

#accuracy
print("accuracy_score: ",metrics.accuracy_score(label_4_valid, y_valid_pred))

accuracy_score:  0.9333333333333333


In [49]:
#@title Hyper Parameter testing
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold
from scipy.stats import uniform
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,HalvingGridSearchCV

# Define the hyperparameter grid
param_grid = {
    'C': [0.1,1,10,20,100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Experiment with different kernels
    'gamma':['scale', 'auto'],
    'degree': [1,2,3,4]

}

# Create an SVM classifier
svm_classifier = SVC(class_weight='balanced')

# Perform grid search with cross-validation
grid_search = HalvingGridSearchCV(svm_classifier, param_grid, scoring='accuracy', cv=5,n_jobs=-1, factor=2, verbose=1)
grid_search.fit(x_train_pca, label_4)

# Get the best model and hyperparameters
best_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the validation set
y_pred = best_svm_classifier.predict(x_valid_pca)

# Evaluate the model
print("Best Hyperparameters:", best_params)
print("accuracy_score: ",metrics.accuracy_score(label_4_valid, y_pred))

n_iterations: 8
n_required_iterations: 8
n_possible_iterations: 8
min_resources_: 219
max_resources_: 28040
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 160
n_resources: 219
Fitting 5 folds for each of 160 candidates, totalling 800 fits
----------
iter: 1
n_candidates: 80
n_resources: 438
Fitting 5 folds for each of 80 candidates, totalling 400 fits
----------
iter: 2
n_candidates: 40
n_resources: 876
Fitting 5 folds for each of 40 candidates, totalling 200 fits
----------
iter: 3
n_candidates: 20
n_resources: 1752
Fitting 5 folds for each of 20 candidates, totalling 100 fits
----------
iter: 4
n_candidates: 10
n_resources: 3504
Fitting 5 folds for each of 10 candidates, totalling 50 fits
----------
iter: 5
n_candidates: 5
n_resources: 7008
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 6
n_candidates: 3
n_resources: 14016
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 7
n_candidates: 2
n_resources:

In [64]:
#@title Making csv for label 4
predicted_label_test = best_svm_classifier.predict(x_test_pca)

predicted_df = pd.DataFrame({'label_4': predicted_label_test})

# Save the DataFrame to a CSV file
predicted_df.to_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label4.csv', index=False)

#Label 2


In [74]:
#@title svm classifier without pca
from sklearn import svm

# For classification with a linear kernel
classifier4 = svm.SVC(kernel='rbf', C=1)

# For regression with a linear kernel
# classifier = svm.SVR(kernel='linear', C=1.0)

# Train the model
# classifier.fit(x_train_df, y_train_df)
classifier4.fit(X, label_2)

#predict for validation
y_valid_pred = classifier4.predict(X_valid)

#accuracy
print("accuracy_score: ",metrics.accuracy_score(label_2_valid, y_valid_pred))

accuracy_score:  0.8804347826086957


In [71]:
#@title svm classifier with pca
from sklearn import svm

# For classification with a linear kernel
classifier2 = svm.SVC(kernel='linear', C=1)

# For regression with a linear kernel
# classifier = svm.SVR(kernel='linear', C=1.0)

# Train the model
# classifier.fit(x_train_df, y_train_df)
classifier2.fit(x_train_pca, label_2)

#predict for validation
y_valid_pred = classifier2.predict(x_valid_pca)

#accuracy
print("accuracy_score: ",metrics.accuracy_score(label_2_valid, y_valid_pred))

accuracy_score:  0.7853260869565217


In [72]:
#@title Hyper parameter tuning
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold
from scipy.stats import uniform
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,HalvingGridSearchCV

# Define the hyperparameter grid
param_grid = {
    'C': [0.1,1,10,20,100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Experiment with different kernels
    'gamma':['scale', 'auto'],
    'degree': [1,2,3,4]

}

# Create an SVM classifier
svm_classifier2 = SVC(class_weight='balanced')

# Perform grid search with cross-validation
grid_search = HalvingGridSearchCV(svm_classifier2, param_grid, scoring='accuracy', cv=5,n_jobs=-1, factor=2, verbose=1)
grid_search.fit(x_train_pca, label_2)

# Get the best model and hyperparameters
best_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the validation set
y_pred = best_svm_classifier.predict(x_valid_pca)

# Evaluate the model
print("Best Hyperparameters:", best_params)
print("accuracy_score: ",metrics.accuracy_score(label_2_valid, y_pred))

n_iterations: 8
n_required_iterations: 8
n_possible_iterations: 8
min_resources_: 219
max_resources_: 28040
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 160
n_resources: 219
Fitting 5 folds for each of 160 candidates, totalling 800 fits
----------
iter: 1
n_candidates: 80
n_resources: 438
Fitting 5 folds for each of 80 candidates, totalling 400 fits
----------
iter: 2
n_candidates: 40
n_resources: 876
Fitting 5 folds for each of 40 candidates, totalling 200 fits
----------
iter: 3
n_candidates: 20
n_resources: 1752
Fitting 5 folds for each of 20 candidates, totalling 100 fits
----------
iter: 4
n_candidates: 10
n_resources: 3504
Fitting 5 folds for each of 10 candidates, totalling 50 fits
----------
iter: 5
n_candidates: 5
n_resources: 7008
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 6
n_candidates: 3
n_resources: 14016
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 7
n_candidates: 2
n_resources:

In [93]:
#@title svm classifier with best score parameters
from sklearn import svm
clas_l2 = svm.SVC(kernel='poly', C=20, gamma='scale', degree=1)

clas_l2.fit(X, label_2)

#predict for validation
y_valid_2 = clas_l2.predict(X_valid)

#accuracy
print("accuracy_score: ",metrics.accuracy_score(label_2_valid, y_valid_2))

#predict
# y_test_pred = classifier.predict(x_test_df)
#y_test_predict_after_pca = classifier.predict(x_test_df_pca)

accuracy_score:  0.8614130434782609


In [94]:
#@title making csv for label 2
lab_2_val = clas_l2.predict(X_test)

pred_df_2 = pd.DataFrame({'label_2': lab_2_val})

# Save the DataFrame to a CSV file
pred_df_2.to_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label2final.csv', index=False)

In [ ]:
# @title SVM hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define the hyperparameter grid
param_grid = {
    'C': [1,10,20],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4],  # For polynomial kernel
    'gamma': ['scale', 'auto'] + [0.1, 1, 10],  # For rbf and sigmoid kernels
    'tol': [1e-4, 1e-3, 1e-2],
    'class_weight': [None, 'balanced']
}

# Create the SVM classifier
svm = SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=3)
grid_search.fit(x_train_pca, label_3)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("best parameters:", best_params)


#Combining CSV


In [86]:
# df1 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label1.csv')
# df2 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label2.csv')
# df3 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label3.csv')
# df4 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label4.csv')

# combined_df = pd.concat([df1, df2, df3, df4], ignore_index=True)
# combined_df.to_csv('/content/drive/MyDrive/ML/Results/Layer7_Final.csv', index=True)

# Read the CSV files into separate DataFrames
# df1 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label1.csv', header=None, names=['label_1'])
# df2 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label2.csv', header=None, names=['label_2'])
# df3 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label3.csv', header=None, names=['label_3'])
# df4 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label4.csv', header=None, names=['label_4'])

# # Combine the DataFrames into one DataFrame using concat
# combined_df = pd.concat([df1, df2, df3, df4], axis=1, ignore_index=True)

# # Add an index column
# combined_df['Index'] = combined_df.index

# # Save the combined DataFrame to a new CSV file
# combined_df.to_csv('/content/drive/MyDrive/ML/Results/Layer7_Final2.csv', index=False)


# import pandas as pd

# Read the CSV files into separate DataFrames
df1 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label2.csv')
df3 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label3.csv')
df4 = pd.read_csv('/content/drive/MyDrive/ML/Results/predicted_L7_label4.csv')


# Combine the DataFrames into one DataFrame with the index as the first column
combined_df = pd.concat([df1, df2, df3, df4], axis=1, ignore_index=False)

# Add an index column and move it to the first position
combined_df.insert(0, 'ID', range(1,len(combined_df)+1))

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('/content/drive/MyDrive/ML/Results/Layer7_Final.csv', index=False)
